In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm
import cv2
import os
import seaborn as sns
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True) 

In [ ]:
# code to find weights for 3 models

from keras.models import load_model
from sklearn.metrics import accuracy_score
import os
import cv2
import numpy as np
import pandas as pd

base_dir = 'C:/brain_tumor/version2/notebook/Testing'
class_names = ['glioma', 'meningioma', 'pituitary']

X = []
y = []

for label, class_name in enumerate(class_names):
    class_dir = os.path.join(base_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

X = X.astype('float32') / 255.0

model1 = load_model('cnn_case_3.h5')
model2 = load_model('resnet_case_3.h5')
model3 = load_model('inception_case_3.h5')

models = [model1, model2, model3]
preds1 = [model.predict(X) for model in models]
preds1 = np.array(preds1)

df = pd.DataFrame([])

for w1 in range(0, 5):
    for w2 in range(0, 5):
        for w3 in range(0, 5):
            wts = [w1/10., w2/10., w3/10.]
            wted_preds1 = np.tensordot(preds1, wts, axes=((0), (0)))
            wted_ensemble_pred = np.argmax(wted_preds1, axis=1)
            weighted_accuracy = accuracy_score(y, wted_ensemble_pred)
            df = pd.concat([df, pd.DataFrame({'wt1': [wts[0]], 'wt2': [wts[1]], 
                                              'wt3': [wts[2]], 'acc': [weighted_accuracy*100]})], ignore_index=True)

max_acc_row = df.iloc[df['acc'].idxmax()]
print("Max accuracy of ", max_acc_row['acc'], " obtained with w1=", max_acc_row['wt1'],
      " w2=", max_acc_row['wt2'], " and w3=", max_acc_row['wt3'])


In [ ]:
# view all combination of weights

df_sorted = df.sort_values(by='acc', ascending=False)
print(df_sorted)

Ensemble model classify 1 img

In [ ]:
from keras.models import load_model
import numpy as np
import cv2

model1 = load_model('cnn_case_3.h5')
model2 = load_model('resnet_case_3.h5')
model3 = load_model('inception_case_3.h5')

w1, w2, w3 = 0.2, 0.4, 0.3  

new_image_path = 'C:/brain_tumor/version2/classify_dataset/meningioma/meni (8).jpg'
img = cv2.imread(new_image_path)
img = cv2.resize(img, (128, 128))
img = np.array(img)
img = img.astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

pred1 = model1.predict(img)
pred2 = model2.predict(img)
pred3 = model3.predict(img)

combined_pred = (w1 * pred1) + (w2 * pred2) + (w3 * pred3)

final_class = np.argmax(combined_pred, axis=1)

class_labels = ['glioma', 'meningioma', 'pituitary']

predicted_label = class_labels[final_class[0]]
print(f'The predicted tumor type is: {predicted_label}')
